In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
# !pip uninstall -y tensorflow

In [1]:
# !pip3 install numba cupy

In [3]:
# with tf.device(tf.DeviceSpec(device_type="gpu")):
#     pass

In [4]:
# !pip install tensorflow-gpu==2.8.0
# !pip install protobuf==3.20

In [5]:
import tensorflow as tf

In [6]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
# print("Hub Version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "non available")

Version:  2.8.0
Eager mode:  True
GPU is available


2023-03-26 12:56:17.263032: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-26 12:56:17.277819: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-26 12:56:17.278585: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [7]:
"""Version:  2.0.0
Eager mode:  True
Hub Version:  0.7.0
GPU is available"""

'Version:  2.0.0\nEager mode:  True\nHub Version:  0.7.0\nGPU is available'

In [8]:
# !pip install --upgrade intel-extension-for-tensorflow[gpu]

In [9]:
tf.DeviceSpec(device_type="gpu")

In [11]:
%%time
# with tf.device(tf.DeviceSpec(device_type="cpu")):
with tf.device('cpu'):

    sdg = tf.keras.optimizers.SGD(learning_rate=0.01)
    rmsprop = tf.keras.optimizers.RMSprop(learning_rate=0.01)
    adam = tf.keras.optimizers.Adam(learning_rate=0.01)

    x = tf.Variable(25.0, dtype=tf.float32)
    y = tf.Variable(25.0, dtype=tf.float32)
    z = tf.Variable(25.0, dtype=tf.float32)

    fn_target = lambda: x**2+y**2+2*z

    i = 0

    with tf.GradientTape() as g:
        g.watch(x)
        g.watch(y)
        g.watch(z)

        t = fn_target()
        gradients = g.gradient(t, sources=[x,y,z])

    #     sdg.apply_gradients(zip(gradients, [x,y]))

        while abs(fn_target() - 0) >= 0.01:
            i = i + 1
            n = sdg.minimize(fn_target, var_list=[x,y,z])
        
print(f"x={x.numpy()} y={y.numpy()} z={z.numpy()}")
# print(x,y)
print(i)

x=2.694846845674448e-10 y=2.694846845674448e-10 z=-0.0003815889358520508
1250
CPU times: user 1.24 s, sys: 0 ns, total: 1.24 s
Wall time: 1.24 s


In [2]:
from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer   

# normal function to run on cpu
def func(a):                                
    for i in range(10000000):
        a[i]+= 1      

# function optimized to run on gpu 
@jit(target_backend='cuda')                         
def func2(a):
    for i in range(10000000):
        a[i]+= 1
if __name__=="__main__":
    n = 10000000                            
    a = np.ones(n, dtype = np.float64)
    
    start = timer()
    func(a)
    print("without GPU:", timer()-start)    
    
    start = timer()
    func2(a)
    print("with GPU:", timer()-start)

ModuleNotFoundError: No module named 'numba'